In [ ]:
#colab
# google-drive-ocamlfuseのインストール
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Colab用のAuth token作成
from google.colab import auth
auth.authenticate_user()

# Drive FUSE library用のcredential生成
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# drive/ を作り、そこにGoogle Driveをマウントする
!mkdir -p drive
!google-drive-ocamlfuse drive

In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [ ]:
import numpy as np
import pandas as pd
import time
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

#Preprocessing

split data set on train and test

In [ ]:
train = pd.read_csv('drive/app/ToxicComments/train.csv', quotechar='"').fillna(' ')

# print train.describe()

#razdelqme gi na train i test (~20%)
test = train.iloc[130000:,:]
train = train.iloc[:130000,:]
train_text = train['comment_text']
test_text = test['comment_text']

In [ ]:
len(test)

1grams

In [ ]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(train_text)

train_features = word_vectorizer.transform(train_text)
test_features = word_vectorizer.transform(test_text)


In [ ]:
train_features


In [ ]:
word_vectorizer2 = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(2, 2),
    max_features=10000)
word_vectorizer2.fit(train_text)

train_features_2grams = word_vectorizer2.transform(train_text)
test_features_2grams = word_vectorizer2.transform(test_text)

In [ ]:
word_vectorizer3 = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(3, 3),
    max_features=10000)
word_vectorizer3.fit(train_text)

train_features_3grams = word_vectorizer3.transform(train_text)
test_features_3grams = word_vectorizer3.transform(test_text)

#Logistic Regression

In [ ]:
st= time.time()
scores = []
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(C=0.1, solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='accuracy'))
    scores.append(cv_score)
    print 'Score for class '+str(class_name)+': '+str(cv_score)

    classifier.fit(train_features, train_target)
    
print('Metrics for 1gram')
print ('Total score: '+str(np.mean(scores)))
print ('Total time: '+str(int(time.time()-st)))

In [ ]:
st= time.time()
scores = []
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(C=0.1, solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_features_2grams , train_target , cv=3, scoring='accuracy'))
    scores.append(cv_score)
    print 'Score for class '+str(class_name)+': '+str(cv_score)

    classifier.fit(train_features_2grams, train_target)
    
print('Metrics for _2grams ')
print ('Total score: '+str(np.mean(scores)))
print ('Total time: '+str(int(time.time()-st)))

In [ ]:
st= time.time()
scores = []
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(C=0.1, solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_features_3grams, train_target, cv=3, scoring='accuracy'))
    scores.append(cv_score)
    print 'Score for class '+str(class_name)+': '+str(cv_score)

    classifier.fit(train_features_3grams, train_target)
    
print('Metrics for _3grams ')
print ('Total score: '+str(np.mean(scores)))
print ('Total time: '+str(int(time.time()-st)))

#Naive Bayes

In [ ]:
st= time.time()
scores = []
for class_name in class_names:
    train_target = train[class_name]
    classifier = MultinomialNB()

    cv_score = np.mean(cross_val_score(classifier, train_features.toarray(), train_target.as_matrix(), cv=3, scoring='accuracy'))
    scores.append(cv_score)
    print 'Score for class '+str(class_name)+': '+str(cv_score)

    classifier.fit(train_features.toarray(), train_target.as_matrix())
    
print('Metrics for _1grams ')
print ('Total score: '+str(np.mean(scores)))
print ('Total time: '+str(int(time.time()-st)))

In [ ]:
st= time.time()
scores = []
for class_name in class_names:
    train_target = train[class_name]
    classifier = MultinomialNB()

    cv_score = np.mean(cross_val_score(classifier, train_features_2grams.toarray(), train_target.as_matrix(), cv=3, scoring='accuracy'))
    scores.append(cv_score)
    print 'Score for class '+str(class_name)+': '+str(cv_score)

    classifier.fit(train_features_2grams.toarray(), train_target.as_matrix())
    
print('Metrics for _2grams ')
print ('Total score: '+str(np.mean(scores)))
print ('Total time: '+str(int(time.time()-st)))

In [ ]:
st= time.time()
scores = []
for class_name in class_names:
    train_target = train[class_name]
    classifier = MultinomialNB()

    cv_score = np.mean(cross_val_score(classifier, train_features_3grams.toarray(), train_target.as_matrix(), cv=3, scoring='accuracy'))
    scores.append(cv_score)
    print 'Score for class '+str(class_name)+': '+str(cv_score)

    classifier.fit(train_features_3grams.toarray(), train_target.as_matrix())
    
print('Metrics for _3grams ')
print ('Total score: '+str(np.mean(scores)))
print ('Total time: '+str(int(time.time()-st)))

#Random Forest

In [ ]:
st= time.time()
scores = []
for class_name in class_names:
    train_target = train[class_name]
    classifier = RandomForestClassifier(n_estimators=50, max_depth=50, random_state=1)

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='accuracy'))
    scores.append(cv_score)
    print 'Score for class '+str(class_name)+': '+str(cv_score)

    classifier.fit(train_features, train_target)

print('Metrics for _1grams ')
print 'Total score: '+str(np.mean(scores))
print 'Total time: '+str(int(time.time()-st))

In [ ]:
st= time.time()
scores = []
for class_name in class_names:
    train_target = train[class_name]
    classifier = RandomForestClassifier(n_estimators=50, max_depth=50, random_state=1)

    cv_score = np.mean(cross_val_score(classifier, train_features_2grams, train_target, cv=3, scoring='accuracy'))
    scores.append(cv_score)
    print 'Score for class '+str(class_name)+': '+str(cv_score)

    classifier.fit(train_features_2grams, train_target)

print('Metrics for _2grams ')
print 'Total score: '+str(np.mean(scores))
print 'Total time: '+str(int(time.time()-st))

In [ ]:
st= time.time()
scores = []
for class_name in class_names:
    train_target = train[class_name]
    classifier = RandomForestClassifier(n_estimators=50, max_depth=50, random_state=1)

    cv_score = np.mean(cross_val_score(classifier, train_features_3grams, train_target, cv=3, scoring='accuracy'))
    scores.append(cv_score)
    print 'Score for class '+str(class_name)+': '+str(cv_score)

    classifier.fit(train_features_2grams, train_target)

print('Metrics for _3grams ')
print 'Total score: '+str(np.mean(scores))
print 'Total time: '+str(int(time.time()-st))

#Neural Network

In [ ]:
st= time.time()
scores = []
for class_name in class_names:
    train_target = train[class_name]
    classifier = MLPClassifier(solver='adam', learning_rate='adaptive', hidden_layer_sizes=(50, 5), random_state=1, batch_size=5000) #, max_iter=100)

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='accuracy'))
    scores.append(cv_score)
    print 'Score for class '+str(class_name)+': '+str(cv_score)

    classifier.fit(train_features, train_target)
print('Metrics for _1grams ')
print 'Total score: '+str(np.mean(scores))
print 'Total time: '+str(int(time.time()-st))

In [ ]:
st= time.time()
scores = []
for class_name in class_names:
    train_target = train[class_name]
    classifier = MLPClassifier(solver='adam', learning_rate='adaptive', hidden_layer_sizes=(50, 5), random_state=1, batch_size=5000) #, max_iter=100)

    cv_score = np.mean(cross_val_score(classifier, train_features_2grams, train_target, cv=3, scoring='accuracy'))
    scores.append(cv_score)
    print 'Score for class '+str(class_name)+': '+str(cv_score)

    classifier.fit(train_features_2grams, train_target)
print('Metrics for _2grams ')
print 'Total score: '+str(np.mean(scores))
print 'Total time: '+str(int(time.time()-st))

In [ ]:
st= time.time()
scores = []
for class_name in class_names:
    train_target = train[class_name]
    classifier = MLPClassifier(solver='adam', learning_rate='adaptive', hidden_layer_sizes=(50, 5), random_state=1, batch_size=5000) #, max_iter=100)

    cv_score = np.mean(cross_val_score(classifier, train_features_3grams, train_target, cv=3, scoring='accuracy'))
    scores.append(cv_score)
    print 'Score for class '+str(class_name)+': '+str(cv_score)

    classifier.fit(train_features_3grams, train_target)
print('Metrics for _3grams ')
print 'Total score: '+str(np.mean(scores))
print 'Total time: '+str(int(time.time()-st))

#Convolution Neural Network

In [ ]:
import keras
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [ ]:
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K


In [ ]:

max_features=10000
nb_filter = 250
filter_length = 3
hidden_dims = 250
nb_epoch = 1
nb_classes=6
batch_size=200



In [ ]:
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K


In [ ]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, dropout=0.2))
# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))
def max_1d(X):
    return K.max(X, axis=1)

model.add(Lambda(max_1d, output_shape=(nb_filter,)))#maks pooling
model.add(Dense(hidden_dims)) 
model.add(Dropout(0.2)) 
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
train_target=train.drop(['id','comment_text'], axis=1)
test_target=test.drop(['id','comment_text'], axis=1)

In [ ]:
st= time.time()
print('Train...')
model.fit(train_features, train_target, batch_size=batch_size, nb_epoch=1,
          validation_data=(test_features, test_target))
score, acc = model.evaluate(test_features, test_target,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)
print 'Total time: '+str(int(time.time()-st))

print("Generating test predictions...")
preds = model.predict_classes(test_features, verbose=0)


In [ ]:
print("Generating test predictions...")
preds = model.predict_classes(test_features, verbose=0)